Iñigo Martin Melero

## Analisis Random Sample Amat

### 0) Librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier

np.random.seed(51)

### 1) Preprocesamiento

In [2]:
df1=pd.read_csv('Bancarrota_Total_nd.csv',encoding = 'unicode_escape')
print(df1)

        ID  Coef1_Springate  Coef2_Springate  Coef3_Springate  \
0        1           0.9525           0.0062           0.0054   
1        2           0.6894           0.1641           0.3116   
2        3           0.8671          -0.0479          -0.5388   
3        4          -0.0034           0.0296          -0.2114   
4        5           0.2916           0.0341           0.0534   
...    ...              ...              ...              ...   
1440  1502          -0.0555          -0.3837          -0.3051   
1441  1503           0.0061          -0.4382          -1.0570   
1442  1505           0.8625         -24.6623          -6.1772   
1443  1511          -2.2275          -0.0105          -0.0031   
1444  1512          -0.9579          -0.1787          -0.0904   

      Coef4_Springate  Coef1_Amat  Coef2_Amat  Coef3_Amat  Coef4_Amat  \
0              3.9499      1.0056      0.0064     -0.0150     -2.3367   
1              2.8739      1.7204      0.4234      0.1196      0.2824   


In [3]:
df1.dtypes

ID                   int64
Coef1_Springate    float64
Coef2_Springate    float64
Coef3_Springate    float64
Coef4_Springate    float64
Coef1_Amat         float64
Coef2_Amat         float64
Coef3_Amat         float64
Coef4_Amat         float64
Resultado            int64
dtype: object

In [4]:
df2=pd.read_csv('Activa_Total_nd.csv',encoding = 'unicode_escape')
print(df2)

            ID  Coef1_Springate  Coef2_Springate  Coef3_Springate  \
0            1          -0.1786           0.0810           0.2086   
1            2           0.3473           0.0039           0.1439   
2            3           0.4718           0.0385           0.0752   
3            4           0.1373           0.0490           0.0493   
4            5           0.0656           0.0826           0.5767   
...        ...              ...              ...              ...   
675939  732517           0.1232          -0.1647          -5.9479   
675940  732532           0.1989          -2.1463          -0.7942   
675941  732550          -0.0927           0.0000           0.0208   
675942  732570           0.6813          -0.0064          -0.0240   
675943  732590           0.4244          -0.0416          -0.0402   

        Coef4_Springate  Coef1_Amat  Coef2_Amat  Coef3_Amat  Coef4_Amat  \
0                2.3157      0.9028      0.6267      0.0598      0.0954   
1                1.89

In [5]:
df2.dtypes

ID                   int64
Coef1_Springate    float64
Coef2_Springate    float64
Coef3_Springate    float64
Coef4_Springate    float64
Coef1_Amat         float64
Coef2_Amat         float64
Coef3_Amat         float64
Coef4_Amat         float64
Resultado            int64
dtype: object

In [6]:
df2=df2.sample(n=1445)
df2

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
509245,525415,0.0329,0.0301,0.0608,0.3828,0.9728,0.1153,0.0217,0.1877,2
44975,45353,0.3456,0.1565,153.4201,0.9415,847.6308,0.9315,0.1424,0.1529,2
456318,468018,0.3334,0.0418,0.0224,1.3175,0.4216,-0.8659,0.0418,-0.0483,2
565824,588917,0.4336,-0.1111,-0.0618,1.0781,0.3493,-1.0970,-0.1133,0.1033,2
429119,438988,0.3327,0.6120,1.2551,1.4989,1.5065,0.5125,0.4772,0.9312,2
...,...,...,...,...,...,...,...,...,...,...
231861,234389,0.2465,0.1972,0.6429,0.9455,0.9694,0.5755,0.1484,0.2578,2
400655,408888,0.4364,-0.3206,-0.6137,1.5450,0.9059,0.0536,-0.3180,-5.9348,2
18991,19185,0.4876,0.0416,0.0642,2.4778,1.8070,0.2396,0.0207,0.0864,2
359551,365780,0.0689,0.1701,0.6630,0.5095,2.1370,0.5045,0.1198,0.2376,2


### 2) Preparacion datos para ML


In [7]:
df3=pd.concat([df1, df2])
df3=df3.reset_index(drop=True)
df3

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
0,1,0.9525,0.0062,0.0054,3.9499,1.0056,0.0064,-0.0150,-2.3367,1
1,2,0.6894,0.1641,0.3116,2.8739,1.7204,0.4234,0.1196,0.2824,1
2,3,0.8671,-0.0479,-0.5388,2.0795,0.4260,-1.4245,-1.1645,0.8174,1
3,4,-0.0034,0.0296,-0.2114,0.6265,0.4756,0.2109,-0.1652,-0.7833,1
4,5,0.2916,0.0341,0.0534,0.3098,0.9662,0.3709,0.0299,0.0807,1
...,...,...,...,...,...,...,...,...,...,...
2885,234389,0.2465,0.1972,0.6429,0.9455,0.9694,0.5755,0.1484,0.2578,2
2886,408888,0.4364,-0.3206,-0.6137,1.5450,0.9059,0.0536,-0.3180,-5.9348,2
2887,19185,0.4876,0.0416,0.0642,2.4778,1.8070,0.2396,0.0207,0.0864,2
2888,365780,0.0689,0.1701,0.6630,0.5095,2.1370,0.5045,0.1198,0.2376,2


In [8]:
df3 = df3.sample(frac=1).reset_index(drop=True)
df3

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
0,18150,0.6890,-0.0089,-0.0473,2.1577,3.0430,0.7209,-0.0090,-0.0124,2
1,486889,0.6038,0.2531,1.3658,1.4567,5.0709,0.8171,0.1872,0.2292,2
2,108009,0.1331,0.3404,1.1933,1.3505,2.1191,0.4382,0.2577,0.5880,2
3,416619,0.4063,0.0140,0.0024,0.8539,0.7983,0.0921,0.0011,0.0123,2
4,478748,-0.6579,0.0732,0.0198,4.6421,0.3548,-3.4860,0.0517,-0.0148,2
...,...,...,...,...,...,...,...,...,...,...
2885,377942,-0.0579,0.0780,0.1796,1.3015,0.1996,0.1370,0.0543,0.3963,2
2886,1299,0.0487,-53.2081,-1.5327,159.3218,0.0013,-114.9521,-56.4382,0.4910,1
2887,72539,0.4744,0.0311,0.1484,0.3603,4.5046,0.6370,0.0287,0.0451,2
2888,212848,0.2383,0.0730,0.1669,1.3356,2.4502,0.5988,0.0484,0.0807,2


In [9]:
#GridSearchCV utiliza validación cruzada, y luego nosotros haremos también validación cruzada con nuestros datos, por lo que
#a GridSearchCV no le metemos solamente el Training Set, le metemos TODO. 
X=df3.drop(columns=["Resultado","ID"])
Y=df3.drop(columns=["ID","Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat","Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])

In [10]:
X

,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat
0,0.6890,-0.0089,-0.0473,2.1577,3.0430,0.7209,-0.0090,-0.0124
1,0.6038,0.2531,1.3658,1.4567,5.0709,0.8171,0.1872,0.2292
2,0.1331,0.3404,1.1933,1.3505,2.1191,0.4382,0.2577,0.5880
3,0.4063,0.0140,0.0024,0.8539,0.7983,0.0921,0.0011,0.0123
4,-0.6579,0.0732,0.0198,4.6421,0.3548,-3.4860,0.0517,-0.0148
...,...,...,...,...,...,...,...,...
2885,-0.0579,0.0780,0.1796,1.3015,0.1996,0.1370,0.0543,0.3963
2886,0.0487,-53.2081,-1.5327,159.3218,0.0013,-114.9521,-56.4382,0.4910
2887,0.4744,0.0311,0.1484,0.3603,4.5046,0.6370,0.0287,0.0451
2888,0.2383,0.0730,0.1669,1.3356,2.4502,0.5988,0.0484,0.0807


In [11]:
Y

,Resultado
0,2
1,2
2,2
3,2
4,2
...,...
2885,2
2886,1
2887,2
2888,2


### 3.1) Valores Test de Springate

Calculamos si está comprometida la situación de la empresa según Amat.

Su formula es:

Z = - 0.862 + 1.03 x Coef1 + 3.07 x Coef2 + 0.66 x Coef3 + 0.4 x Coef4

Z positivo empresa bien, Z negativo empresa mal.

Cogemos otro Dataframe, y multiplicamos primero y luego sumamos.

In [12]:
formula_Spr=X
formula_Spr=formula_Spr.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat"])
formula_Spr.loc[:,'Coef1_Springate'] *= 1.03
formula_Spr.loc[:,'Coef2_Springate'] *= 3.07
formula_Spr.loc[:,'Coef3_Springate'] *= 0.66
formula_Spr.loc[:,'Coef4_Springate'] *= 0.4
formula_Spr["constant"]=-0.862
formula_Spr["Z"] = formula_Spr.sum(axis=1)

formula_Spr["Y_pred_Springate"] = 2
formula_Spr.loc[formula_Spr["Z"] <= 0, "Y_pred_Springate"] = 1
Y_pred_formula_Spr=formula_Spr["Y_pred_Springate"]
print("Accuracy: "+str(accuracy_score(Y, Y_pred_formula_Spr)))
print("F1 score: "+str(f1_score(Y, Y_pred_formula_Spr)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred_formula_Spr)))
print("Confusion matrix:")
confusion_matrix(Y, Y_pred_formula_Spr)

Accuracy: 0.6422145328719723
F1 score: 0.6629726205997393
Kappa: 0.28442906574394466
Confusion matrix:


array([[1017,  428],
       [ 606,  839]], dtype=int64)

### 3.2) Valores Test de Grover

Su formula es:

Z = + 0.057 + 0.02 + 1.650 x Coef1 + 3.404 x Coef2 - 0.016 x Coef3

Z positivo empresa bien, Z negativo empresa mal.

Cogemos otro Dataframe, y multiplicamos primero y luego sumamos.

In [13]:
formula_Gro=X
formula_Gro=formula_Gro.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Springate","Coef4_Springate","Coef4_Amat"])
formula_Gro.loc[:,'Coef1_Springate'] *= 1.65
formula_Gro.loc[:,'Coef2_Springate'] *= 3.404
formula_Gro.loc[:,'Coef3_Amat'] *= -0.016
formula_Gro["constant"]=0.077
formula_Gro["Z"] = formula_Gro.sum(axis=1)

formula_Gro["Y_pred_Grover"] = 2
formula_Gro.loc[formula_Gro["Z"] <= 0, "Y_pred_Grover"] = 1
Y_pred_formula_Gro=formula_Gro["Y_pred_Grover"]
print("Accuracy: "+str(accuracy_score(Y, Y_pred_formula_Gro)))
print("F1 score: "+str(f1_score(Y, Y_pred_formula_Gro)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred_formula_Gro)))
print("Confusion matrix:")
confusion_matrix(Y, Y_pred_formula_Gro)

Accuracy: 0.6695501730103807
F1 score: 0.5899527694289395
Kappa: 0.3391003460207612
Confusion matrix:


array([[ 687,  758],
       [ 197, 1248]], dtype=int64)

### 3.3) Valores Test de Amat

Calculamos si está comprometida la situación de la empresa según Amat. 

Su formula es:

Z = - 3.9 + 1.28 x Coef1 + 6.1 x Coef2 + 6.5 x Coef3 + 4.8 x Coef4

Z positivo empresa bien, Z negativo empresa mal.

Cogemos otro Dataframe, y multiplicamos primero y luego sumamos.

In [14]:
formula_Ama=X
formula_Ama=formula_Ama.drop(columns=["Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])
formula_Ama.loc[:,'Coef1_Amat'] *= 1.28
formula_Ama.loc[:,'Coef2_Amat'] *= 6.1
formula_Ama.loc[:,'Coef3_Amat'] *= 6.5
formula_Ama.loc[:,'Coef4_Amat'] *= 4.8
formula_Ama["constant"]=-3.9
formula_Ama["Z"] = formula_Ama.sum(axis=1)

formula_Ama["Y_pred_Amat"] = 2
formula_Ama.loc[formula_Ama["Z"] <= 0, "Y_pred_Amat"] = 1
Y_pred_formula_Ama=formula_Ama["Y_pred_Amat"]
print("Accuracy: "+str(accuracy_score(Y, Y_pred_formula_Ama)))
print("F1 score: "+str(f1_score(Y, Y_pred_formula_Ama)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred_formula_Ama)))
print("Confusion matrix:")
confusion_matrix(Y, Y_pred_formula_Ama)

Accuracy: 0.6795847750865052
F1 score: 0.6757703081232492
Kappa: 0.35916955017301033
Confusion matrix:


array([[965, 480],
       [446, 999]], dtype=int64)

### 3.4) Combinacion de predicciones

In [15]:
labels_total=pd.concat([Y,Y_pred_formula_Spr, Y_pred_formula_Gro,Y_pred_formula_Ama], axis=1)
print(labels_total.to_string())

      Resultado  Y_pred_Springate  Y_pred_Grover  Y_pred_Amat
0             2                 2              2            2
1             2                 2              2            2
2             2                 2              2            2
3             2                 1              2            1
4             2                 2              1            1
5             2                 1              1            1
6             2                 2              2            2
7             2                 2              2            2
8             1                 1              2            1
9             1                 1              1            1
10            1                 2              2            1
11            2                 1              2            2
12            2                 1              2            1
13            2                 2              2            2
14            2                 2              2            2
15      

In [16]:
bancarrota=labels_total.loc[(labels_total['Resultado']==1)]

In [17]:
at_least_1_bancarrota=sum(bancarrota.sum(axis=1)<7)
print(at_least_1_bancarrota)
all_match_bancarrota=sum(bancarrota.sum(axis=1)==4)
print(all_match_bancarrota)

1226
534


### 4) Modelos ML

### 4.1) Estudio con Springate

In [18]:
X_Springate=X.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat"])
X_Springate

,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate
0,0.6890,-0.0089,-0.0473,2.1577
1,0.6038,0.2531,1.3658,1.4567
2,0.1331,0.3404,1.1933,1.3505
3,0.4063,0.0140,0.0024,0.8539
4,-0.6579,0.0732,0.0198,4.6421
...,...,...,...,...
2885,-0.0579,0.0780,0.1796,1.3015
2886,0.0487,-53.2081,-1.5327,159.3218
2887,0.4744,0.0311,0.1484,0.3603
2888,0.2383,0.0730,0.1669,1.3356


K Nearest Neighbours

In [19]:
grid_params={"n_neighbors":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
             "weights": ["uniform","distance"],
             "metric":["minkowski","euclidean","manhattan"],
             "algorithm":["auto","ball_tree","kd_tree","brute"]}
gs=GridSearchCV(KNeighborsClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Springate,Y)
gs_res.best_params_

Fitting 10 folds for each of 480 candidates, totalling 4800 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


{'algorithm': 'auto',
 'metric': 'minkowski',
 'n_neighbors': 2,
 'weights': 'uniform'}

In [20]:
knn = KNeighborsClassifier(n_neighbors=2,weights="uniform",metric="minkowski",algorithm="auto")
Y_pred=cross_val_predict(knn, X_Springate, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.6294117647058823
F1 score: 0.6903729401561145
Kappa: 0.2588235294117647


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1

array([[1194,  251],
       [ 820,  625]], dtype=int64)

Random Forest

In [21]:
grid_params={"bootstrap":[True,False],
             "max_features":["auto","sqrt"],
             "max_depth": [10,20,30,40,50],
             "n_estimators":[200,400,600,800,1000]
            }

gs=GridSearchCV(RandomForestClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Springate, Y)
gs_res.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 200}

In [22]:
rf = RandomForestClassifier(bootstrap= True, max_depth=10, max_features="auto", n_estimators= 200)
Y_pred=cross_val_predict(rf, X_Springate, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\

Accuracy: 0.7269896193771627
F1 score: 0.7261367580701145
Kappa: 0.4539792387543252


array([[1046,  399],
       [ 390, 1055]], dtype=int64)

Decision Tree

In [23]:
grid_params={"criterion":["gini","entropy","log_loss"],
             "splitter":["best","random"],
             "max_depth": [10,20,30,40,50,60,70,80,90,100],
            }

gs=GridSearchCV(tree.DecisionTreeClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Springate, Y)
gs_res.best_params_

Fitting 10 folds for each of 60 candidates, totalling 600 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
200 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\tree\_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_C

{'criterion': 'entropy', 'max_depth': 10, 'splitter': 'best'}

In [24]:
clf = tree.DecisionTreeClassifier(criterion= "entropy", max_depth=10, splitter="best")
Y_pred=cross_val_predict(clf, X_Springate, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.6892733564013841
F1 score: 0.6943498978897209
Kappa: 0.3785467128027682


array([[1020,  425],
       [ 473,  972]], dtype=int64)

Naive Bayes

In [25]:
grid_params={'var_smoothing': np.logspace(0,-9, num=100),
            }

gs=GridSearchCV(GaussianNB(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Springate, Y)
gs_res.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'var_smoothing': 1.519911082952933e-07}

In [26]:
gnb = GaussianNB(var_smoothing= 0.000000152)
Y_pred=cross_val_predict(gnb, X_Springate, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.514878892733564
F1 score: 0.07519788918205804
Kappa: 0.02975778546712804


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array w

array([[  57, 1388],
       [  14, 1431]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [27]:
grid_params={'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
            'activation': ['tanh', 'relu','identity','logistic'],
            'solver': ['sgd', 'adam','lbfgs'],
            'alpha': [0.00001,0.0001,0.001,0.01,0.1],
            'learning_rate': ['constant','adaptive','invscaling'],
            }

gs=GridSearchCV(MLPClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Springate, Y)
gs_res.best_params_

Fitting 10 folds for each of 540 candidates, totalling 5400 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'activation': 'logistic',
 'alpha': 0.001,
 'hidden_layer_sizes': (50, 100, 50),
 'learning_rate': 'constant',
 'solver': 'sgd'}

In [28]:
mlp = MLPClassifier(activation="logistic",alpha=0.001,hidden_layer_sizes=(50,100,50),learning_rate="constant",solver="sgd")
Y_pred=cross_val_predict(mlp, X_Springate, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_p

Accuracy: 0.4996539792387543
F1 score: 0.4996539792387543
Kappa: -0.0006920415224913601


array([[722, 723],
       [723, 722]], dtype=int64)

Linear Discriminant Analysis

In [29]:
grid_params={'solver': ['svd', 'lsqr', 'eigen'], 
             } 

gs=GridSearchCV(LinearDiscriminantAnalysis(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Springate, Y)
gs_res.best_params_

Fitting 10 folds for each of 3 candidates, totalling 30 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'solver': 'svd'}

In [30]:
lda = LinearDiscriminantAnalysis(solver="svd")
Y_pred=cross_val_predict(lda, X_Springate, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.5474048442906574
F1 score: 0.32298136645962733
Kappa: 0.09480968858131489


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array w

array([[ 312, 1133],
       [ 175, 1270]], dtype=int64)

## 4.2) Estudio con Grover

In [31]:
X_Grover=X.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Springate","Coef4_Springate","Coef4_Amat"])
X_Grover

,Coef1_Springate,Coef2_Springate,Coef3_Amat
0,0.6890,-0.0089,-0.0090
1,0.6038,0.2531,0.1872
2,0.1331,0.3404,0.2577
3,0.4063,0.0140,0.0011
4,-0.6579,0.0732,0.0517
...,...,...,...
2885,-0.0579,0.0780,0.0543
2886,0.0487,-53.2081,-56.4382
2887,0.4744,0.0311,0.0287
2888,0.2383,0.0730,0.0484


K Nearest Neighbours

In [32]:
grid_params={"n_neighbors":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
             "weights": ["uniform","distance"],
             "metric":["minkowski","euclidean","manhattan"],
             "algorithm":["auto","ball_tree","kd_tree","brute"]}
gs=GridSearchCV(KNeighborsClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Grover,Y)
gs_res.best_params_

Fitting 10 folds for each of 480 candidates, totalling 4800 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


{'algorithm': 'auto',
 'metric': 'manhattan',
 'n_neighbors': 2,
 'weights': 'uniform'}

In [33]:
knn = KNeighborsClassifier(n_neighbors=2,weights="uniform",metric="manhattan",algorithm="auto")
Y_pred=cross_val_predict(knn, X_Grover, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.6238754325259516
F1 score: 0.6773523300682694
Kappa: 0.24775086505190314


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1

array([[1141,  304],
       [ 783,  662]], dtype=int64)

Random Forest

In [34]:
grid_params={"bootstrap":[True,False],
             "max_features":["auto","sqrt"],
             "max_depth": [10,20,30,40,50],
             "n_estimators":[200,400,600,800,1000]
            }

gs=GridSearchCV(RandomForestClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Grover, Y)
gs_res.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'n_estimators': 200}

In [35]:
rf = RandomForestClassifier(bootstrap= True, max_depth=10, max_features="sqrt", n_estimators= 200)
Y_pred=cross_val_predict(rf, X_Grover, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\

Accuracy: 0.7235294117647059
F1 score: 0.7169677647892312
Kappa: 0.44705882352941173


array([[1012,  433],
       [ 366, 1079]], dtype=int64)

Decision Tree

In [36]:
grid_params={"criterion":["gini","entropy","log_loss"],
             "splitter":["best","random"],
             "max_depth": [10,20,30,40,50,60,70,80,90,100],
            }

gs=GridSearchCV(tree.DecisionTreeClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Grover, Y)
gs_res.best_params_

Fitting 10 folds for each of 60 candidates, totalling 600 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
200 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\tree\_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_C

{'criterion': 'gini', 'max_depth': 10, 'splitter': 'best'}

In [37]:
clf = tree.DecisionTreeClassifier(criterion= "gini", max_depth=10, splitter="best")
Y_pred=cross_val_predict(clf, X_Grover, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.7017301038062284
F1 score: 0.6966924700914848
Kappa: 0.4034602076124567


array([[ 990,  455],
       [ 407, 1038]], dtype=int64)

Naive Bayes

In [38]:
grid_params={'var_smoothing': np.logspace(0,-9, num=100),
            }

gs=GridSearchCV(GaussianNB(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Grover, Y)
gs_res.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'var_smoothing': 1e-06}

In [39]:
gnb = GaussianNB(var_smoothing= 0.000001)
Y_pred=cross_val_predict(gnb, X_Grover, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.5204152249134948
F1 score: 0.08935611038107753
Kappa: 0.04083044982698958


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array w

array([[  68, 1377],
       [   9, 1436]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [40]:
grid_params={'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
            'activation': ['tanh', 'relu','identity','logistic'],
            'solver': ['sgd', 'adam','lbfgs'],
            'alpha': [0.00001,0.0001,0.001,0.01,0.1],
            'learning_rate': ['constant','adaptive','invscaling'],
            }

gs=GridSearchCV(MLPClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Grover, Y)
gs_res.best_params_

Fitting 10 folds for each of 540 candidates, totalling 5400 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'activation': 'relu',
 'alpha': 0.0001,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'invscaling',
 'solver': 'sgd'}

In [41]:
mlp = MLPClassifier(activation="relu",alpha=0.0001,hidden_layer_sizes=(100,),learning_rate="invscaling",solver="sgd")
Y_pred=cross_val_predict(mlp, X_Grover, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_p

Accuracy: 0.5653979238754325
F1 score: 0.35523613963039014
Kappa: 0.13079584775086506


array([[ 346, 1099],
       [ 157, 1288]], dtype=int64)

Linear Discriminant Analysis

In [42]:
grid_params={'solver': ['svd', 'lsqr', 'eigen'], 
             } 

gs=GridSearchCV(LinearDiscriminantAnalysis(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Grover, Y)
gs_res.best_params_

Fitting 10 folds for each of 3 candidates, totalling 30 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'solver': 'svd'}

In [43]:
lda = LinearDiscriminantAnalysis(solver="svd")
Y_pred=cross_val_predict(lda, X_Grover, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.5470588235294118
F1 score: 0.24553314121037464
Kappa: 0.09411764705882353


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array w

array([[ 213, 1232],
       [  77, 1368]], dtype=int64)

## 4.3) Estudio con Amat

In [44]:
X_Amat=X.drop(columns=["Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])
X_Amat

,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat
0,3.0430,0.7209,-0.0090,-0.0124
1,5.0709,0.8171,0.1872,0.2292
2,2.1191,0.4382,0.2577,0.5880
3,0.7983,0.0921,0.0011,0.0123
4,0.3548,-3.4860,0.0517,-0.0148
...,...,...,...,...
2885,0.1996,0.1370,0.0543,0.3963
2886,0.0013,-114.9521,-56.4382,0.4910
2887,4.5046,0.6370,0.0287,0.0451
2888,2.4502,0.5988,0.0484,0.0807


K Nearest Neighbours

In [45]:
grid_params={"n_neighbors":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
             "weights": ["uniform","distance"],
             "metric":["minkowski","euclidean","manhattan"],
             "algorithm":["auto","ball_tree","kd_tree","brute"]}
gs=GridSearchCV(KNeighborsClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Amat,Y)
gs_res.best_params_

Fitting 10 folds for each of 480 candidates, totalling 4800 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


{'algorithm': 'auto',
 'metric': 'manhattan',
 'n_neighbors': 2,
 'weights': 'uniform'}

In [46]:
knn = KNeighborsClassifier(n_neighbors=2,weights="uniform",metric="manhattan",algorithm="auto")
Y_pred=cross_val_predict(knn, X_Amat, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.6740484429065744
F1 score: 0.721301775147929
Kappa: 0.3480968858131488


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1

array([[1219,  226],
       [ 716,  729]], dtype=int64)

Random Forest

In [47]:
grid_params={"bootstrap":[True,False],
             "max_features":["auto","sqrt"],
             "max_depth": [10,20,30,40,50],
             "n_estimators":[200,400,600,800,1000]
            }

gs=GridSearchCV(RandomForestClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Amat, Y)
gs_res.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'n_estimators': 600}

In [48]:
rf = RandomForestClassifier(bootstrap= True, max_depth=10, max_features="sqrt", n_estimators= 600)
Y_pred=cross_val_predict(rf, X_Amat, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\

Accuracy: 0.740484429065744
F1 score: 0.7413793103448275
Kappa: 0.48096885813148793


array([[1075,  370],
       [ 380, 1065]], dtype=int64)

Decision Tree

In [49]:
grid_params={"criterion":["gini","entropy","log_loss"],
             "splitter":["best","random"],
             "max_depth": [10,20,30,40,50,60,70,80,90,100],
            }

gs=GridSearchCV(tree.DecisionTreeClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Amat, Y)
gs_res.best_params_

Fitting 10 folds for each of 60 candidates, totalling 600 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
200 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\tree\_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_C

{'criterion': 'entropy', 'max_depth': 20, 'splitter': 'random'}

In [50]:
clf = tree.DecisionTreeClassifier(criterion= "entropy", max_depth=20, splitter="random")
Y_pred=cross_val_predict(clf, X_Amat, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.6910034602076125
F1 score: 0.6942827798699075
Kappa: 0.3820069204152249


array([[1014,  431],
       [ 462,  983]], dtype=int64)

Naive Bayes

In [51]:
grid_params={'var_smoothing': np.logspace(0,-9, num=100),
            }

gs=GridSearchCV(GaussianNB(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Amat, Y)
gs_res.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'var_smoothing': 0.12328467394420659}

In [52]:
gnb = GaussianNB(var_smoothing= 0.1232)
Y_pred=cross_val_predict(gnb, X_Amat, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.5024221453287198
F1 score: 0.1726121979286536
Kappa: 0.0048442906574394096


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array w

array([[ 150, 1295],
       [ 143, 1302]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [53]:
grid_params={'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
            'activation': ['tanh', 'relu','identity','logistic'],
            'solver': ['sgd', 'adam','lbfgs'],
            'alpha': [0.00001,0.0001,0.001,0.01,0.1],
            'learning_rate': ['constant','adaptive','invscaling'],
            }

gs=GridSearchCV(MLPClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Amat, Y)
gs_res.best_params_

Fitting 10 folds for each of 540 candidates, totalling 5400 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'activation': 'logistic',
 'alpha': 0.001,
 'hidden_layer_sizes': (50, 100, 50),
 'learning_rate': 'adaptive',
 'solver': 'sgd'}

In [54]:
mlp = MLPClassifier(activation="logistic",alpha=0.001,hidden_layer_sizes=(50,100,50),learning_rate="adaptive",solver="sgd")
Y_pred=cross_val_predict(mlp, X_Amat, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_p

Accuracy: 0.4996539792387543
F1 score: 0.4996539792387543
Kappa: -0.0006920415224913601


array([[722, 723],
       [723, 722]], dtype=int64)

Linear Discriminant Analysis

In [55]:
grid_params={'solver': ['svd', 'lsqr', 'eigen'], 
             } 

gs=GridSearchCV(LinearDiscriminantAnalysis(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X_Amat, Y)
gs_res.best_params_

Fitting 10 folds for each of 3 candidates, totalling 30 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'solver': 'svd'}

In [56]:
lda = LinearDiscriminantAnalysis(solver="svd")
Y_pred=cross_val_predict(lda, X_Amat, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.5951557093425606
F1 score: 0.5806451612903226
Kappa: 0.19031141868512114


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array w

array([[810, 635],
       [535, 910]], dtype=int64)

## 4.4) Estudio con todo

K Nearest Neighbours

In [57]:
grid_params={"n_neighbors":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
             "weights": ["uniform","distance"],
             "metric":["minkowski","euclidean","manhattan"],
             "algorithm":["auto","ball_tree","kd_tree","brute"]}
gs=GridSearchCV(KNeighborsClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X,Y)
gs_res.best_params_

Fitting 10 folds for each of 480 candidates, totalling 4800 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


{'algorithm': 'auto',
 'metric': 'minkowski',
 'n_neighbors': 2,
 'weights': 'uniform'}

In [58]:
knn = KNeighborsClassifier(n_neighbors=2,weights="uniform",metric="minkowski",algorithm="auto")
Y_pred=cross_val_predict(knn, X, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.6570934256055363
F1 score: 0.7065442700621855
Kappa: 0.3141868512110727


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1

array([[1193,  252],
       [ 739,  706]], dtype=int64)

Random Forest

In [59]:
grid_params={"bootstrap":[True,False],
             "max_features":["auto","sqrt"],
             "max_depth": [10,20,30,40,50],
             "n_estimators":[200,400,600,800,1000]
            }

gs=GridSearchCV(RandomForestClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X, Y)
gs_res.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 1000}

In [60]:
rf = RandomForestClassifier(bootstrap= True, max_depth=10, max_features="auto", n_estimators= 1000)
Y_pred=cross_val_predict(rf, X, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:1044: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\

Accuracy: 0.7508650519031141
F1 score: 0.7501734906315058
Kappa: 0.5017301038062284


array([[1081,  364],
       [ 356, 1089]], dtype=int64)

Decision Tree

In [61]:
grid_params={"criterion":["gini","entropy","log_loss"],
             "splitter":["best","random"],
             "max_depth": [10,20,30,40,50,60,70,80,90,100],
            }

gs=GridSearchCV(tree.DecisionTreeClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X, Y)
gs_res.best_params_

Fitting 10 folds for each of 60 candidates, totalling 600 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
200 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\tree\_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_C

{'criterion': 'gini', 'max_depth': 10, 'splitter': 'best'}

In [62]:
clf = tree.DecisionTreeClassifier(criterion= "gini", max_depth=10, splitter="best")
Y_pred=cross_val_predict(clf, X, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.7145328719723183
F1 score: 0.7167868177136972
Kappa: 0.4290657439446367


array([[1044,  401],
       [ 424, 1021]], dtype=int64)

Naive Bayes

In [63]:
grid_params={'var_smoothing': np.logspace(0,-9, num=100),
            }

gs=GridSearchCV(GaussianNB(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X, Y)
gs_res.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'var_smoothing': 1.519911082952933e-07}

In [64]:
gnb = GaussianNB(var_smoothing= 0.000000152)
Y_pred=cross_val_predict(gnb, X, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.5249134948096886
F1 score: 0.11017498379779649
Kappa: 0.04982698961937715


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array w

array([[  85, 1360],
       [  13, 1432]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [65]:
grid_params={'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
            'activation': ['tanh', 'relu','identity','logistic'],
            'solver': ['sgd', 'adam','lbfgs'],
            'alpha': [0.00001,0.0001,0.001,0.01,0.1],
            'learning_rate': ['constant','adaptive','invscaling'],
            }

gs=GridSearchCV(MLPClassifier(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X, Y)
gs_res.best_params_

Fitting 10 folds for each of 540 candidates, totalling 5400 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'activation': 'logistic',
 'alpha': 0.1,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'invscaling',
 'solver': 'sgd'}

In [66]:
mlp = MLPClassifier(activation="logistic",alpha=0.1,hidden_layer_sizes=(100,),learning_rate="invscaling",solver="sgd")
Y_pred=cross_val_predict(mlp, X, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_p

Accuracy: 0.4982698961937716
F1 score: 0.5334620334620335
Kappa: -0.0034602076124568004


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[829, 616],
       [834, 611]], dtype=int64)

Linear Discriminant Analysis

In [67]:
grid_params={'solver': ['svd', 'lsqr', 'eigen'], 
             } 

gs=GridSearchCV(LinearDiscriminantAnalysis(),grid_params,verbose=1,cv=10,n_jobs=-1,scoring="recall")
gs_res=gs.fit(X, Y)
gs_res.best_params_

Fitting 10 folds for each of 3 candidates, totalling 30 fits


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'solver': 'svd'}

In [68]:
lda = LinearDiscriminantAnalysis(solver="svd")
Y_pred=cross_val_predict(lda, X, Y, cv=10)
print("Accuracy: "+str(accuracy_score(Y, Y_pred)))
print("F1 score: "+str(f1_score(Y, Y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y, Y_pred)))
conf_mat = confusion_matrix(Y, Y_pred)
conf_mat

Accuracy: 0.5972318339100346
F1 score: 0.5025641025641026
Kappa: 0.19446366782006919


C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array w

array([[ 588,  857],
       [ 307, 1138]], dtype=int64)